In [14]:
# Import the modules
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [15]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path('../Credit_Risk/Resources/lending_data.csv')

lending_df = pd.read_csv(file_path)

# Review the DataFrame
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


In [16]:
# Checking the balance of each class 
lending_df['loan_status'].value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [17]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df['loan_status']

# Separate the X variable, the features
X = lending_df.drop(columns='loan_status')

In [18]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [19]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Split the data into training and testing datasets by using `train_test_split`.

In [20]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [21]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
model = LogisticRegression(solver='lbfgs',
                           max_iter=200,
                           random_state=1)

# Fit the model using training data
model.fit(X_train, y_train)

# Check model scores
print(f'Training Data Score: {model.score(X_train, y_train)}')
print(f'Testing Data Score: {model.score(X_test, y_test)}')

Training Data Score: 0.9914878250103177
Testing Data Score: 0.9924164259182832


### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [22]:
# Make a prediction using the testing data
y_predictions = model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.

* Print the classification report.

In [23]:
# Generate a confusion matrix for the model
matrix = confusion_matrix(y_test, y_predictions )

print(matrix)

[[18679    80]
 [   67   558]]


In [24]:
# Print the classification report for the model
label_names = ['healthy loan (0)', 'high-risk loan (1)']

report = classification_report(y_test, y_predictions, target_names=label_names)

print(report)

                    precision    recall  f1-score   support

  healthy loan (0)       1.00      1.00      1.00     18759
high-risk loan (1)       0.87      0.89      0.88       625

          accuracy                           0.99     19384
         macro avg       0.94      0.94      0.94     19384
      weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** 

How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 

Overall the model has a general accuracy of 99%, but given the imbalanced classes within the dataset (75,036 healthy loans vs only 2,500 high-risk loans), lets looks at our specific categories for a deeper understanding of the model...


**Predicting a Healthy Loan:**

The model predicts the 'healthy loan' category with a precision of 100%, meaning that every loan that the model predicted as 'healthy' was indeed a healthy loan. 

The model also had a success rate of 100% in predicting all the *actual* healthy loans in the test data as indicated by the recall score of 1.0. So the model correctly identified all healthy loans from our test data. 



**Predicting a High-Risk Loan:**

Given the context of the data, the real metric we are interested in is predicting credit risk, so this is where the analysis needs to be thorough. In the context of lending, incorrectly classifying a high-risk loan as healthy could result in significant financial losses for the institution. Hence, this category warrants detailed scrutiny.

The model predicts the 'high-risk loan' category with a precision of 87%, meaning that 87% of the loans that the model predicted as high-risk, were in fact high-risk. 

The model correctly predicted the *actual* high-risk loans with a success rate of 89% (as indicated by the recall score of 0.89). 

<span style="color:red">**This means that the model missed 11% of high-risk loans. From our test data, this equates to *67 missed high-risk loans out of a total 625 high-risk loans* **</span>

An F1-score of 88% for the high-risk loan category means that the model has a balanced performance in terms of both false positives and false negatives for this category.

**Conclusion:**

Given the recall of `89%` for high-risk loans, this model seems effective in identifying a majority of potential credit risks. However, as with all models, continual monitoring and adjustment might be needed to ensure its performance over time


---